### As bibliotecas importadas serão utilizadas para:
- Manipulação de dados (pandas, numpy)
- Visualizações (matplotlib, seaborn)
- Divisão e validação de dados (train_test_split, cross_val_score)
- Métricas de avaliação (classification_report, roc_auc_score, confusion_matrix)
- Modelo de classificação (GradientBoostingClassifier)
- Persistência do modelo (joblib)

### Imports utilizados:
- **pandas**: Manipulação e análise de dados
- **numpy**: Operações numéricas e arrays
- **matplotlib.pyplot**: Criação de visualizações e gráficos
- **seaborn**: Visualizações estatísticas de alto nível
- **train_test_split**: Divisão dos dados em conjuntos de treino e teste
- **cross_val_score**: Validação cruzada para avaliar modelos
- **classification_report, roc_auc_score, confusion_matrix**: Métricas de avaliação para classificação
- **GradientBoostingClassifier**: Modelo de classificação baseado em gradient boosting
- **joblib**: Serialização e salvamento de modelos

#### Objetivo deste código é realizar a modelagem de dados utilizando um modelo de classificação, avaliar seu desempenho e salvar o modelo treinado para uso futuro.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

### Variavel da base de dados para o modelo

In [ ]:
import os
BD_MODELO = os.getenv("BD_MODELO","../parquet/refined/refined.parquet")

### Carregamento e Análise Exploratória

#### Objetivo
Carregar o dataset final tratado e realizar análise exploratória inicial.

#### Explicação
- read_parquet: carrega base otimizada e refinada
- head(): visualiza primeiras linhas
- info(): verifica tipos de dados
- describe(): estatísticas descritivas
- isnull().sum(): checa valores ausentes

Aqui garantimos que os dados estão consistentes antes da modelagem.

In [ ]:

df = pd.read_parquet(BD_MODELO)
df.head()
df.info()
df.describe()
df.isnull().sum()

### Criação da Variável Target

#### Objetivo
Transformar o problema em *classificação binária*

#### Explicação
- Se defasagem != 0 → 1 (com risco)
- Se defasagem == 0 → 0 (sem risco)

#### Variaveis 
X: variáveis explicativas
y: variável alvo

- Removemos a variável original para evitar vazamento de dados (data leakage).


In [ ]:
df['defasagem_binaria'] = (df['defasagem'] != 0).astype(int)
X = df.drop(columns=['defasagem'])
X = X.drop(columns=['defasagem_binaria'])
y = df['defasagem_binaria']


### Distribuição da Variável Target

#### Objetivo
Verificar balanceamento da base.

#### Explicação
Analisa proporção de alunos com e sem defasagem.

Isso é importante para:
- Escolher métricas adequadas
- Definir estratégia de validação
- Ajustar threshold



In [ ]:
y.value_counts(normalize=True)

### Divisão Treino/Teste

#### Objetivo
Separar dados para validação do modelo.

#### Explicação
- 80% treino
- 20% teste
- stratify=y: mantém proporção das classes
- random_state=42: reprodutibilidade
- Garante avaliação justa e robusta.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### Criação e Treinamento do Modelo

#### Objetivo
Criar modelo de classificação baseado em Gradient Boosting.

#### Explicação dos hiperparâmetros:
- n_estimators=500: número de árvores
- learning_rate=0.05: taxa de aprendizado
- max_depth=4: profundidade das árvores
- random_state=42: reprodutibilidade

In [ ]:
model = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    random_state=42
)

### Treinamento:

In [ ]:
model.fit(X_train, y_train)

### Predição probabilística:
📌 Utilizamos probabilidades para permitir ajuste de threshold.

In [ ]:
y_proba = model.predict_proba(X_test)[:,1]

### Ajuste de Threshold

#### Objetivo
Controlar sensibilidade do modelo.

#### Explicação
Ao invés de usar threshold padrão (0.5), foi definido 0.35 para:
- Aumentar recall
- Reduzir falsos negativos
- Priorizar identificação de alunos em risco
- Estratégia adequada para problemas sociais onde perder um aluno em risco é mais crítico que gerar falso positivo.

In [ ]:
threshold = 0.35
y_pred = (y_proba >= threshold).astype(int)


### Avaliação do Modelo

#### Métricas Avaliadas:
- Precision
- Recall
- F1-score
- AUC (Área sob a curva ROC)
- AUC mede capacidade discriminativa geral do modelo.


In [ ]:
print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))

### Matriz de Confusão
Visualização com heatmap.

#### Objetivo
Analisar erros do modelo.

Permite identificar:
- Verdadeiros positivos
- Verdadeiros negativos
- Falsos positivos
- Falsos negativos
- Essencial para decisões estratégicas.


In [ ]:
cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de Confusão")
plt.show()


### Validação Cruzada

#### Objetivo
Avaliar estabilidade do modelo.

#### Explicação
- 5 folds
- Métrica: AUC
- Retorna média e desvio padrão
- Garante que o modelo não está overfitting.



In [ ]:
scores = cross_val_score(
    model,
    X,
    y,
    cv=5,
    scoring='roc_auc'
)

print("AUC médio:", scores.mean())
print("Desvio:", scores.std())


### Correlação com Variável Original

#### Objetivo
Analisar relação entre variáveis e defasagem original.

#### Explicação
Permite identificar:
- Variáveis mais impactantes
- Potenciais insights para políticas educacionais


In [ ]:
print(df.corr()['defasagem'].sort_values(ascending=False))

### Salvamento do Modelo
#### Objetivo
Persistir modelo treinado para uso em API.

#### Explicação
- Serializa modelo
- Permite carregamento em produção
- Fundamental para deploy
- Modelo pronto para integração via Flask/FastAPI.

In [ ]:
import joblib

path_model = os.getenv("PATH_MODEL", "../app/model/modelo_defasagem.joblib")

joblib.dump(model, path_model)

print("Modelo salvo com sucesso!")
